In [1]:
#!pip install aquacrop==0.0.7

In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from aquacrop.core import *
from aquacrop.classes import *

# AquaCrop-OS tutorial: basics

<img src="imgs/crop_soil_system.jpg" width="400" height="350"/> 
http://blogs.oregonstate.edu/inspire/tag/waterwise

## The aquacrop model has 3 components:

1) Climate: rainfall, temperature, evapotranspiration and CO2

2) Soil water balance 

3) Crop growth

## In order to run the model we have to define: climate, soil, crop and time scale

The first run will take ~50s as we use jit compiler to speed up the model; after that a single season should be < 0.5s

In [3]:
wdf = prepare_weather(get_filepath('champion_weather.txt'))
soil = SoilClass('loam')
crop = CropClass('maize')
model = AquaCropModel('2018/05/01','2018/10/31',wdf,soil,crop)
model.initialize()
model.step(till_termination=True)
final = model.Outputs.Final; final

,Season,Name,HarvestDate,Step,Yield,Tirr
0,0,maize,2018-09-16,137,14.060806,0.0


In [4]:
# rerun over multiple years and plot
#final.Yield.plot()

# Weather

The first thing we define is the weather; aquacrop has a number of built in climate files such as from Champion, Nebraska. Others available can be seen with `os.listdir(aquacrop.data.__path__[0])`

In [5]:
wdf = prepare_weather(get_filepath('champion_weather.txt'))
wdf.head()

,MinTemp,MaxTemp,Precipitation,ReferenceET,Date
0,-21.11,3.33,0.0,1.59,1982-01-01
1,-10.00,0.56,0.0,0.86,1982-01-02
2,-11.67,-2.22,0.0,0.72,1982-01-03
3,-12.22,7.22,0.0,1.71,1982-01-04
4,-14.44,-1.11,0.0,0.92,1982-01-05


Of course you can use any climate file as long as it contains daily values of:
- min temperature
- max temperature
- Rainfall
- Reference EvapoTranspiration
- Date

they must have the correct column names as shown above

# Soil

Next we create a soil profile. This can be done simply using one of the built in soil types

In [6]:
soil = SoilClass('clay_loam'); soil.profile

,Comp,Layer,dz,dzsum,zBot,zTop,zMid,th_dry,th_wp,th_fc,th_s,Ksat,penetrability,tau
0,0,1.0,0.1,0.1,0.1,0.0,0.05,0.109,0.218,0.467,0.664,490.9,100,0.76
1,1,1.0,0.1,0.2,0.2,0.1,0.15,0.109,0.218,0.467,0.664,490.9,100,0.76
2,2,1.0,0.1,0.3,0.3,0.2,0.25,0.109,0.218,0.467,0.664,490.9,100,0.76
3,3,1.0,0.1,0.4,0.4,0.3,0.35,0.109,0.218,0.467,0.664,490.9,100,0.76
4,4,1.0,0.1,0.5,0.5,0.4,0.45,0.109,0.218,0.467,0.664,490.9,100,0.76
5,5,1.0,0.1,0.6,0.6,0.5,0.55,0.109,0.218,0.467,0.664,490.9,100,0.76
6,6,1.0,0.2,0.8,0.8,0.6,0.70,0.109,0.218,0.467,0.664,490.9,100,0.76
7,7,1.0,0.2,1.0,1.0,0.8,0.90,0.109,0.218,0.467,0.664,490.9,100,0.76
8,8,1.0,0.2,1.2,1.2,1.0,1.10,0.109,0.218,0.467,0.664,490.9,100,0.76
9,9,1.0,0.2,1.4,1.4,1.2,1.30,0.109,0.218,0.467,0.664,490.9,100,0.76


as we can see above the soil profile is broken up compartments.

Run the cell below to see the souce code for `SoilClass`. There you can see all the default soil types as well as custom paramaters you can specify

In [7]:
#SoilClass??

We can also create our own custom soil profile

In [8]:
soil = SoilClass('custom',dz=[0.1]*8+[0.2]*4)
soil.add_layer(thickness=0.8,thWP=0.1,thFC=0.45,thS=0.55,Ksat=500,penetrability=100) # add layer using hydraulic proeprties
soil.add_layer_from_texture(thickness=0.8,Sand=30,Clay=30,OrgMat=2.5,Penetrability=100) # add layer using texture proeprties
soil.profile

,Comp,Layer,dz,dzsum,zBot,zTop,zMid,th_dry,th_wp,th_fc,th_s,Ksat,penetrability,tau
0,0,1.0,0.1,0.1,0.1,0.0,0.05,0.0500,0.100,0.450,0.55,500.0,100.0,0.76
1,1,1.0,0.1,0.2,0.2,0.1,0.15,0.0500,0.100,0.450,0.55,500.0,100.0,0.76
2,2,1.0,0.1,0.3,0.3,0.2,0.25,0.0500,0.100,0.450,0.55,500.0,100.0,0.76
3,3,1.0,0.1,0.4,0.4,0.3,0.35,0.0500,0.100,0.450,0.55,500.0,100.0,0.76
4,4,1.0,0.1,0.5,0.5,0.4,0.45,0.0500,0.100,0.450,0.55,500.0,100.0,0.76
5,5,1.0,0.1,0.6,0.6,0.5,0.55,0.0500,0.100,0.450,0.55,500.0,100.0,0.76
6,6,1.0,0.1,0.7,0.7,0.6,0.65,0.0500,0.100,0.450,0.55,500.0,100.0,0.76
7,7,1.0,0.1,0.8,0.8,0.7,0.75,0.0500,0.100,0.450,0.55,500.0,100.0,0.76
8,8,2.0,0.2,1.0,1.0,0.8,0.90,0.0955,0.191,0.445,0.63,430.0,100.0,0.72
9,9,2.0,0.2,1.2,1.2,1.0,1.10,0.0955,0.191,0.445,0.63,430.0,100.0,0.72


## Crop

Next we need a crop. Again lets look at the source code to see what we can do (warning there is a lot of paramaters we can define)


As an example lets change the planting date to October 1st

In [14]:
crop = CropClass('wheat',PlantingDate='10/1')

In [16]:
crop.PlantingDate, crop.HarvestDate

('10/1', '04/30')

In [17]:
crop.Zmax

1.5

In [18]:
soil.zSoil

1.6

In [30]:
crop.__dict__

{'Name': 'wheat',
 'CropFilename': '',
 'IrrigationFilename': '',
 'FieldMngtFilename': '',
 'fshape_b': 13.8135,
 'PctZmin': 70,
 'fshape_ex': -6,
 'ETadj': 1,
 'Aer': 5,
 'LagAer': 3,
 'beta': 12,
 'a_Tr': 1,
 'GermThr': 0.2,
 'CCmin': 0.05,
 'MaxFlowPct': 33.333333333333336,
 'HIini': 0.01,
 'bsted': 0.000138,
 'bface': 0.001165,
 'CropType': 3,
 'PlantMethod': 1,
 'CalendarType': 2,
 'SwitchGDD': 0,
 'PlantingDate': '10/1',
 'HarvestDate': '04/30',
 'Emergence': 150,
 'MaxRooting': 864,
 'Senescence': 1700,
 'Maturity': 2400,
 'HIstart': 1250,
 'Flowering': 200,
 'YldForm': 1100,
 'GDDmethod': 3,
 'Tbase': 0,
 'Tupp': 26,
 'PolHeatStress': 1,
 'Tmax_up': 35,
 'Tmax_lo': 40,
 'PolColdStress': 1,
 'Tmin_up': 5,
 'Tmin_lo': 0,
 'TrColdStress': 1,
 'GDD_up': 14,
 'GDD_lo': 0,
 'Zmin': 0.3,
 'Zmax': 1.5,
 'fshape_r': 1.5,
 'SxTopQ': 0.048,
 'SxBotQ': 0.012,
 'SeedSize': 1.5,
 'PlantPop': 4500000,
 'CCx': 0.96,
 'CDC': 0.004,
 'CGC': 0.005001,
 'Kcb': 1.1,
 'fage': 0.15,
 'WP': 15,
 'WPy

putting it all together

In [19]:
model = AquaCropModel('2000/10/01','2001/04/30',wdf,soil,crop)
model.initialize()
model.step(till_termination=True)
final = model.Outputs.Final; final

KeyboardInterrupt: 